In [1]:
from pathlib import Path
import numpy as np
import re
from math import prod
from collections import defaultdict, deque
from copy import copy
from itertools import pairwise, permutations, product

In [2]:
data = Path('../Data/Day21.txt').read_text().splitlines()

In [3]:
# Big ideas

# Keep moves in the same direction together

# Because our encode assumes start position is at A
# We can always segment our encode for each segment ending in A

# Codes always end in A
# Shortest length = sum of shortest segments
# We can perform that calculation recursively with memoization

# DO NOT TRY TO GET STRING AT 25 ROBOS, IT'S WAY TOO LONG AND WILL FREEZE YOUR PC
# DON'T ASK ME HOW I KNOW

In [4]:
numpad = {
    '7': (0, 0),
    '8': (0, 1),
    '9': (0, 2),
    '4': (1, 0),
    '5': (1, 1),
    '6': (1, 2),
    '1': (2, 0),
    '2': (2, 1),
    '3': (2, 2),
    '0': (3, 1),
    'A': (3, 2)
}

dirpad = {
    '^': (0, 1),
    'A': (0, 2),
    '<': (1, 0),
    'v': (1, 1),
    '>': (1, 2)
}



def get_sequences(pad, forbidden):

    dr = {
        '^': (-1, 0),
        '>': (0, 1),
        'v': (1, 0),
        '<': (0, -1)
    }
    
    moves = {}

    for num, (i, j) in pad.items():
        for num2, (i2, j2) in pad.items():

            di = i2 - i
            dj = j2 - j

            up = '^'*abs(di)*(di < 0)
            down = 'v'*abs(di)*(di > 0)

            right = '>'*abs(dj)*(dj > 0)
            left = '<'*abs(dj)*(dj < 0)

            # Better to keep directions together
            # rather than mix
            # E.g. >>^^ and ^^>> are fine
            # But >^>^ is not
            sequence = [up, down, right, left]

            moves[num, num2] = set()

            for p in permutations(sequence):
                p = ''.join(p)
                if p in moves[num, num2]:
                    continue

                valid = True

                i3 = i
                j3 = j

                for move in p:
                    
                    di2, dj2 = dr[move]

                    i3 += di2
                    j3 += dj2

                    if (i3, j3) == forbidden:
                        valid = False
                        break
                if valid:
                    moves[num, num2].add(''.join(p))
    
    return moves
            

In [5]:
num_to_num = get_sequences(numpad, (3, 0))
dir_to_dir = get_sequences(dirpad, (0, 0))

In [6]:
def numcode_to_dircodes(code, num_to_num = num_to_num):
    return {'A'.join(p)+'A' for p in product(*[num_to_num[num1, num2] for num1, num2 in pairwise('A'+code)])}

cache = {}

def dircode_to_dircodes(code, dir_to_dir = dir_to_dir, cache = cache):

    if code in cache:
        return cache[code]

    if code.find('A') == len(code)-1:
        cache[code] = {'A'.join(p)+'A' for p in product(*[dir_to_dir[d1, d2] for d1, d2 in pairwise('A'+code)])}
        return cache[code]
    
    return {''.join(p) for p in product(*[dircode_to_dircodes(seg + 'A') for seg in code.split('A')[:-1]])}

In [7]:
shortest_cache = {}

def find_shortest(code, layers, cache = shortest_cache):

    if code == 'A':
        return 1

    if (code, layers) in cache:
        return cache[code, layers]
    
    elif layers == 1:
        cache[code, layers] = len(min(dircode_to_dircodes(code), key = len))

    
    elif code.index('A') == len(code) - 1:
        codes = dircode_to_dircodes(code)
        cache[code, layers] = min([find_shortest(c, layers - 1) for c in codes])

    else:
        cache[code, layers] = sum(find_shortest(seg + 'A', layers) for seg in code.split('A')[:-1])

    return cache[code, layers]

In [8]:
sum([min(find_shortest(code2, 2) for code2 in numcode_to_dircodes(code))*int(code[:-1]) for code in data])

132532

In [9]:
sum([min(find_shortest(code2, 25) for code2 in numcode_to_dircodes(code))*int(code[:-1]) for code in data])

165644591859332